In [1]:
from collections import namedtuple
from math import inf, prod
from Crypto.Util.number import inverse
from typing import List
from sympy import factorint, isprime, prime, integer_nthroot, mod_inverse
from sympy.polys.polytools import invert
from sympy import symbols, Poly, div

Point = namedtuple("Point", "x y isinf", defaults=[False])
def pt_to_str(p: Point):
    return "Point("+str(p.x)+","+str(p.y)+")" if not p.isinf else "O"
Point.__str__ = pt_to_str

EllipticCurve = namedtuple("EllipticCurve", "p a b")

def ec_frobenius(curve: EllipticCurve, P: Point):
    """Apply the Frobenius map to point P = (x, y) on the curve."""
    return Point(pow(P.x, curve.p, curve.p), pow(P.y, curve.p, curve.p), P.isinf)

def ec_neg(curve: EllipticCurve, p: Point):
    return Point(p.x, -p.y % curve.p, p.isinf)

def ec_add(curve: EllipticCurve, p1: Point, p2: Point):
    assert not (p1.isinf and p2.isinf), "cannot add O to O"
    if p1.isinf: return p2
    if p2.isinf: return p1
    if (p1.x==p2.x and p1.y==-p2.y): return Point(0,0,isinf=True)
    if p1.x!=p2.x:
        # print(p2.x, p1.x, curve.p)
        tangent = (p2.y - p1.y)*inverse(p2.x - p1.x, curve.p) # pow(p2.x - p1.x, curve.p-2, curve.p)
    else:
        tangent = (3*(p1.x)**2 + curve.a)*inverse(2*p1.y, curve.p) # pow(2*p1.y, curve.p-2, curve.p)
    p3x = tangent**2 - p1.x - p2.x
    p3y = tangent*(p1.x - p3x) - p1.y
    return Point(p3x%curve.p, p3y%curve.p)

def ec_mul(curve: EllipticCurve, p: Point, n: int):
    if n==0: return Point(p.x, p.y, True)
    p1 = p
    p2 = Point(0,0,True)
    while (n>0):
        if n%2==1: p2 = ec_add(curve, p1, p2)
        p1 = ec_add(curve, p1, p1)
        n = n//2
    return p2

In [2]:
class DivisionPolynomialsList:
    def __init__(self, curve_polynomials):
        class DivisionPolynomials(curve_polynomials):
            def __str__(self):
                return "psi[{0}]".format(
                    self.leading_coefficient().remainder()
                )
        self.__curve_polynomials = DivisionPolynomials

        # __psi is the cache of division polynomials.
        self.__psi = None

    def __getitem__(self, index):
        index = int(index)
        if index < 0:
            raise IndexError

        self.__generate_up_to(index)
        return self.__psi[index]

    def curve_polynomials(self):
        return self.__curve_polynomials

    def __generate_up_to(self, l):
        if not self.__psi:
            self.__psi = 5 * [None]

            # R = F[x,y] / (y**2 - x**3 - A*x - B)
            R = self.__curve_polynomials
            # The polynomial y (used in the recursion scheme)
            self.__y = R(0, 1)

            A, B = self.__curve_polynomials.curve().parameters()

            # Lists are copied by reference; assigning to psi modifies self.__psi
            psi = self.__psi
            psi[0] = R(0, 0)
            psi[1] = R(1, 0)
            psi[2] = R(0, 2)
            psi[3] = R((-(A**2), 12 * B, 6 * A, 0, 3), 0)
            psi[4] = R(
                0,
                (-4 * (8 * (B**2) + A**3), -16 * A *
                 B, -20 * (A**2), 80 * B, 20 * A, 0, 4)
            )

        psi = self.__psi
        y = self.__y
        for j in range(len(self.__psi), l + 1):
            k, m = divmod(j, 2)
            if m:
                # j is odd
                psi.append(psi[k + 2] * psi[k]**3 - psi[k + 1]**3 * psi[k - 1])
            else:
                if k % 2 == 0:
                    psi.append(
                        (psi[k].y_factor() // 2)
                        * (psi[k + 2] * psi[k - 1]**2 - psi[k - 2] * psi[k + 1]**2)
                    )
                else:
                    psi.append(
                        y * (psi[k].x_factor() // 2)
                        * (psi[k + 2] * psi[k - 1].y_factor()**2
                           - psi[k - 2] * psi[k + 1].y_factor()**2)
                    )